In [1]:
# 변수 선언

MODEL_NAME = "skt/kogpt2-base-v2"
DATA_IN_PATH = "./datasets/"
DATA_OUT_PATH = "./models/"
TRAIN_DATA_FILE = "processed_slogan_final.csv"
TRAIN_DATA_NAME = "processed_slogan_final"
PLOT_OUT_PATH = "./plots/"



In [2]:
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [3]:
SPECIAL_TOKENS_DICT  = {
    'eos_token':'</s>',
    'pad_token':'<pad>',
    'additional_special_tokens':['<unused0>', '<unused1>'],  # 설명, 슬로건 컬럼을 나누기 위해 kogpt2의 미사용 토큰을 특수토큰으로 추가 (미사용 토큰을 사용하여 모델 임베딩 크기 조절이 필요없음)
}
tokenizer.add_special_tokens(SPECIAL_TOKENS_DICT)

print(tokenizer.special_tokens_map)

{'eos_token': '</s>', 'pad_token': '<pad>', 'additional_special_tokens': "['<unused0>', '<unused1>']"}


In [4]:
# # 한정된 GPU 메모리를 최대로 활용하기 위한, 데이터 길이 확인
# import csv
# import pandas as pd
# import matplotlib.pyplot as plt
# plt.style.use('seaborn')

# filename =  DATA_IN_PATH + TRAIN_DATA_FILE

# context_tkn = tokenizer.additional_special_tokens_ids[0]  # 토크나이저의 additional_special_tokens_ids[0] : <unused0>
# slogan_tkn = tokenizer.additional_special_tokens_ids[1] # 토크나이저의 additional_special_tokens_ids[1] : <unused1>
# pad_tkn = tokenizer.pad_token_id  #'<pad>'
# eos_tkn = tokenizer.eos_token_id  # </s>

# with open(filename, 'r', encoding="euc-kr") as csvfile:  # 수집된 슬로건 CSV파일 불러오기
#     reader = csv.reader(csvfile)
    
#     # row별 길이를 정보를 담을 빈리스트 생성
#     context_list = []
#     slogan_list = []
#     seq_len_list = []
    
#     for row in reader:
#         context = [context_tkn] + tokenizer.encode(row[0])
#         context_list.append(len(context))
        
#         slogan = [slogan_tkn] + tokenizer.encode(row[1]) + [eos_tkn]
#         slogan_list.append(len(slogan))

#         seq_len = context + slogan
#         seq_len_list.append(len(seq_len))

# # 리스트로 데이터프레임 생성
# scatter_df = pd.DataFrame(context_list, columns=["context"])
# scatter_df['slogan'] = slogan_list
# scatter_df['add_len'] = seq_len_list

# # seaborn box plot을 활용하여 시각화
# red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')

# scatter_df[['context', 'slogan', 'add_len']].plot(kind='box', vert=False, flierprops=red_square, figsize=(16,2));
# plt.show()

In [5]:
import csv
import torch
from torch.utils.data import Dataset

# https://wikidocs.net/57165
# 학습용 데이터 로더

class SloganDataset(Dataset):
  # 데이터셋의 전처리를 해주는 부분
  def __init__(self, filename, tokenizer, seq_length=50): # seq_length : 위의 셀에서 확인한 길이 정보를 토대로 50 지정
    
    context_tkn = tokenizer.additional_special_tokens_ids[0]  # <unused0>
    slogan_tkn = tokenizer.additional_special_tokens_ids[1] # <unused1>
    pad_tkn = tokenizer.pad_token_id  #'<pad>'
    eos_tkn = tokenizer.eos_token_id  # </s>

    self.examples = []  # example 빈리스트 생성
    with open(filename, 'r', encoding="UTF-8") as csvfile:
      reader = csv.reader(csvfile)
      
      for row in reader:
        # 컨텍스트 및 슬로건 세그먼트 구축
        context = [context_tkn] + tokenizer.encode(row[0])
        slogan = [slogan_tkn] + tokenizer.encode(row[1]) + [eos_tkn]
        
        # 두 부분을 함께 연결
        tokens = context + slogan + [pad_tkn] * ( seq_length - len(context) - len(slogan) ) # 50 길이만큼 </pad>토큰 채움

        # 해당 세그먼트로 각 토큰에 주석달기
        segments = [context_tkn] * len(context) + [slogan_tkn] * ( seq_length - len(context) )  

        # 레이블을 -100으로 설정하여 컨텍스트, 패딩 및 <unused1> 토큰을 무시
        labels = [-100] * (len(context)+1) + slogan[1:] + [-100] * ( seq_length - len(context) - len(slogan) )

        # 데이터셋에 전처리된 예제 추가
        self.examples.append((tokens, segments, labels)) #[토큰, 세그먼트, 레이블]
  # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
  # len(dataset)을 했을 때 데이터셋의 크기를 리턴할 len
  def __len__(self):
    return len(self.examples)

  # 데이터셋에서 특정 1개의 샘플을 가져오는 함수
  # dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 get_item
  def __getitem__(self, item):
    return torch.tensor(self.examples[item])


# 데이터세트를 빌드하고 검증을 위해 첫 번째 배치의 차원을 표시
slogan_dataset = SloganDataset(DATA_IN_PATH + TRAIN_DATA_FILE, tokenizer)
print(next(iter(slogan_dataset)).size())

FileNotFoundError: [Errno 2] No such file or directory: './datasets/processed_slogan_final.csv'

In [ ]:
import math, random

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

# 훈련 및 검증 데이터 분할을 위한 인덱스 생성
indices = list(range(len(slogan_dataset)))

random.seed(100) #난수 생성
random.shuffle(indices)

split = math.floor(0.1 * len(slogan_dataset))
train_indices, val_indices = indices[split:], indices[:split]

# PyTorch 데이터 로더를 빌드
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(slogan_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(slogan_dataset, batch_size=64, sampler=val_sampler)


In [ ]:
import numpy as np
from tqdm import tqdm
import os

def fit(model, optimizer, train_dl, val_dl, epochs=1, device=torch.device('cpu')):

  if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    print('--- Directory creation completed successfully ---')
  else:
    print('--- Directory already exists ---')
  
  # epoch당 average training loss를 track
  # epoch당 average validation loss를 track
  avg_train_losses = []
  avg_valid_losses = []
  
  for i in range(epochs):

    print(f'\n--- Starting epoch #{i+1} ---')

    model.train()
    
    # 한 epoch 동안 배치 손실과 배치 크기를 추적을 위한 리스트 생성
    losses = []
    nums = []

    for xb in tqdm(train_dl, desc="Training"):
      # 배치를 훈련 장치로 이동
      inputs = xb.to(device)

      # 토큰 ID, 세그먼트 ID 및 정답(레이블)을 사용하여 모델을 호출
      outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
      
      # 목록에 손실 및 배치 크기를 추가
      loss = outputs[0]
      losses.append(loss.item())
      nums.append(len(xb))

      loss.backward()
      # xm.optimizer_step(optimizer, barrier=True)  # 이부분이 TPU 쓸때 필요한 코드!!
      optimizer.step()
      model.zero_grad()

    # 한 epoch 동안의 평균 비용을 계산
    train_cost = np.sum(np.multiply(losses, nums)) / sum(nums)
    avg_train_losses.append(train_cost)

    # 이제 유효성 검사를 위해 동일한 작업을 수행
    model.eval()
    
    with torch.no_grad():
      losses = []
      nums = []

      for xb in tqdm(val_dl, desc="Validation"):
        inputs = xb.to(device)
        outputs = model(inputs[:,0,:], token_type_ids=inputs[:,1,:], labels=inputs[:,2,:])
        losses.append(outputs[0].item())
        nums.append(len(xb))

    val_cost = np.sum(np.multiply(losses, nums)) / sum(nums)
    avg_valid_losses.append(val_cost)
    
    print(f'\n--- Epoch #{i+1} finished --- Training cost: {train_cost} / Validation cost: {val_cost}')
  
    if (i + 1) % 1 == 0 :
      torch.save(model.state_dict(), DATA_OUT_PATH + TRAIN_DATA_NAME + '_' + f'{i+1}epoch' + '_' + 'model.pth')
      print(f'\n--- Epoch #{i+1} Saving complete ! ---')
      
    torch.cuda.empty_cache()
      
  return avg_train_losses, avg_valid_losses 


In [ ]:
# 이부분이 TPU 쓸때 필요한 코드!! fit 함수확인
# import torch_xla
# import torch_xla.core.xla_model as xm

# # Creates AlexNet for 10 classes
# net = torchvision.models.alexnet(num_classes=10)

# # Acquires the default Cloud TPU core and moves the model to it
# device = xm.xla_device()
# net = net.to(device)

In [ ]:
from transformers import AdamW

# Move the model to the GPU:
device = torch.device('cuda')
model.to(device)

# Fine-tune GPT2 for 5 epochs: 
optimizer = AdamW(model.parameters()) # 트랜스포머의 AdamW
train_loss, valid_loss = fit(model, optimizer, train_loader, val_loader, epochs=10, device=device)

Training:   0%|          | 0/290 [00:00<?, ?it/s]

--- Directory already exists ---

--- Starting epoch #1 ---


Validation: 100%|██████████| 17/17 [00:03<00:00,  5.51it/s]



--- Epoch #1 finished --- Training cost: 7.123012450732287 / Validation cost: 6.4783869582084455


Training:   0%|          | 1/290 [00:00<00:36,  7.93it/s]

...


--- Epoch #9 finished --- Training cost: 3.7935600649142605 / Validation cost: 6.191389684890288


Training:   0%|          | 1/290 [00:00<00:36,  7.93it/s]


--- Epoch #9 Saving complete ! ---

--- Starting epoch #10 ---


Validation: 100%|██████████| 17/17 [00:02<00:00,  5.88it/s]



--- Epoch #10 finished --- Training cost: 3.628651342568955 / Validation cost: 6.267690752175853

--- Epoch #10 Saving complete ! ---


In [ ]:
import matplotlib.pyplot as plt


# 훈련이 진행되는 과정에 따라 loss를 시각화
fig = plt.figure(figsize=(10,8))
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
plt.plot(range(1, len(valid_loss) + 1), valid_loss, label='Validation Loss')

# validation loss의 최저값 지점을 찾기
minposs = valid_loss.index(min(valid_loss)) + 1
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0, 8) # 일정한 scale
plt.xlim(0, len(train_loss)+1) # 일정한 scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

if not os.path.exists(PLOT_OUT_PATH):
    os.makedirs(PLOT_OUT_PATH)
    print('--- Directory creation completed successfully ---')
    
else:
    print('--- Directory already exists ---')

fig.savefig( PLOT_OUT_PATH + TRAIN_DATA_NAME + '_loss_plot.png', bbox_inches = 'tight')